In [1]:
from utz import *
import tabula

[Papermill](https://papermill.readthedocs.io/en/latest/) parameters:

In [ ]:
year = 2019
pages = '33-41'

In [2]:
pdf_path = f'{year}.pdf'
tbls = tabula.read_pdf(pdf_path, pages=pages, multiple_tables=True)
len(tbls)

Got stderr: Jan 14, 2024 1:39:41 PM org.apache.pdfbox.rendering.CIDType0Glyph2D getPathForCharacterCode
Jan 14, 2024 1:39:41 PM org.apache.pdfbox.rendering.CIDType0Glyph2D getPathForCharacterCode
Jan 14, 2024 1:39:41 PM org.apache.pdfbox.rendering.CIDType0Glyph2D getPathForCharacterCode
Jan 14, 2024 1:39:42 PM org.apache.pdfbox.rendering.CIDType0Glyph2D getPathForCharacterCode
Jan 14, 2024 1:39:42 PM org.apache.pdfbox.rendering.CIDType0Glyph2D getPathForCharacterCode



13

In [3]:
def fill_county(tbl):
    t = tbl.copy()
    r0 = t.columns.copy().to_series()
    r0.loc[r0.str.match(r'^Unnamed: \d+$')] = nan
    r0 = r0.to_frame().transpose()
    t.columns = t.iloc[0]
    r0.columns = t.columns
    t = t.iloc[1:]
    t = pd.concat([ r0, t ])
    t = t.drop(index=t[(t == t.columns).all(1)].index)

    # Handle wrapped lines in the 'Persons Killed' column
    col = 'Persons Killed'
    other_cols = [ c for c in t.columns if c != col ]
    wrapped_mask = (~t[col].isna()) & t[other_cols].isna().all(1)
    if wrapped_mask.sum():
        shifted_mask = wrapped_mask.shift(-1).fillna(False)
        before = t.loc[shifted_mask, 'Persons Killed'].copy()
        after = t.loc[shifted_mask, 'Persons Killed'] + ' ' + t['Persons Killed'].shift(-1)[shifted_mask]
        err(f"Handling wrapped 'Persons Killed' lines:")
        err(f"{before}")
        err(f"{after}")
        t.loc[shifted_mask, 'Persons Killed'] = after
        t = t.drop(index=t.index[wrapped_mask])
    
    county_rows = t.isna().sum(1) + 1 == len(t.columns)
    t.loc[county_rows, 'County'] = t[county_rows].apply(lambda r: singleton(r.dropna().values), axis=1)
    t['County'] = t['County'].ffill()
    t = t[~county_rows]
    return t

def process_tbl(tbl):
    t = fill_county(tbl)
    t = sxs(
        t.drop(columns='Time Road'),
        t['Time Road'].str.extract(r'(?P<Time>\d{4}) (?P<Road>.*)'),
    )
    county_matches = t['County'].str.contains(r' COUNTY$', regex=True)
    county_missing = ~county_matches
    assert county_missing.sum() == 0, county_missing
    t['County'] = t['County'].str.replace(' COUNTY$', '', regex=True).str.title()
    t['dt'] = t.apply(lambda r: to_dt(f"{r.Date} {r.Time}"), axis=1)
    t = t.drop(columns=['Date', 'Time'])
    t['Municipality'] = t['Municipality'].str.title()
    t.columns = t.columns.str.lower()
    cols = t.columns.tolist()
    t = t.rename(columns={ 'county': 'cn', 'municipality': 'mn', })
    day = t.day
    assert (t.dt.dt.day_name() == day.str.title()).all()
    cols = ['dt', 'cn', 'mn', 'road', 'mp', 'persons killed', ]
    t = t[cols]
    t['mp'] = t.mp.apply(lambda mp: nan if mp == '--' else float(mp))

    for k in [ 'driver', 'passenger', 'pedestrian', 'pedalcyclist', ]:
        t[k] = t['persons killed'].str.lower().str.extract(f'(?P<{k}>\d+) {k}s?')[k].fillna(0).astype(int)

    t = t.rename(columns={ 'pedalcyclist': 'cyclist', })
    t = t.drop(columns='persons killed')

    return t

In [4]:
df = pd.concat([
    process_tbl(tbl)
    for tbl in tbls
])
df

Handling wrapped 'Persons Killed' lines:
4    1 DRIVER, 1 PASSENGER, 1
Name: Persons Killed, dtype: object
4    1 DRIVER, 1 PASSENGER, 1 PEDESTRIAN
Name: Persons Killed, dtype: object


,dt,cn,mn,road,mp,driver,passenger,pedestrian,cyclist
1,2019-01-13 20:54:00,Atlantic,Egg Harbor Twsp,CR 563,5.6,0,0,1,0
2,2019-01-19 23:55:00,Atlantic,Pleasantville City,SH 40,58.7,0,0,1,0
3,2019-02-03 14:09:00,Atlantic,Hamilton Twsp,HARBOR AVE,NaN,1,0,0,0
4,2019-02-10 18:58:00,Atlantic,Atlantic City,SH 40,62.8,0,0,1,0
5,2019-02-11 23:39:00,Atlantic,Pleasantville City,CR 646,6.5,1,0,0,0
...,...,...,...,...,...,...,...,...,...
5,2019-08-18 18:42:00,Warren,Harmony Twsp,RIVER RD,NaN,1,0,0,0
6,2019-09-03 12:14:00,Warren,Allamuchy Twsp,CR 612,2.6,1,0,0,0
7,2019-10-05 18:57:00,Warren,Blairstown Twsp,I-80,9.9,1,0,0,0
8,2019-10-24 14:14:00,Warren,Liberty Twsp,SH 46,14.9,1,1,0,0


In [5]:
df.to_parquet(f'{year}-crashes.parquet')